In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tik
import pandas as pd
from datetime import datetime, timezone, timedelta

from scipy.stats import norm
import matplotlib.mlab as mlab

from operator import add

#from matplotlib import gridspec
from scipy.optimize import curve_fit

# Input data and log

In [ ]:
#file path and name
filepath = './rawdata/'
filename = 'Feb16_checkEther1.dat'

time_category, error, direction, timercount, reference = np.loadtxt(filepath + filename, comments='#', unpack=True)

In [ ]:
logpath = './log/'
logfilename = 'PMX_2021-02-16.dat'

df = pd.read_csv(logpath + logfilename, delim_whitespace=True, comment='#', skiprows=3322)

In [ ]:
df

In [ ]:
position_log = (df.iloc[:,9][df.iloc[:,9] != "--------"])[(df.iloc[:,9][df.iloc[:,9] != "--------"]) != "0.000"].astype(float).to_numpy()
position_log

In [ ]:
iteration = 224

list_amp = [3.0]
list_intv = ['0.401']

matrix_size = 1

pack_size = 1

In [ ]:
min(error)

In [ ]:
storepath = './plots/'
storename = 'Feb16_checkEther'

## Set variables2

In [ ]:
time = timercount/200e6

plot_slice = 30
capture_slice = 1000

pos_roll = 3

casp_threshold = -100

error_UnixTime = np.array(time_category[np.where((np.diff(timercount) > 110000) | (np.diff(timercount) < 90000))[0]], dtype=int)
error_Time = time[np.where((np.diff(timercount) > 110000) | (np.diff(timercount) < 90000))[0]]

Deg = 360/52000
invDeg = 52000/360

In [ ]:
error_UnixTime

In [ ]:
len(error_UnixTime)

In [ ]:
#first scketch of the rawdata
fig01, axarr01 = plt.subplots(1,2,figsize=(14,7))
ax01 = axarr01.ravel()

ax01[0].plot(time[::100], reference[::100])
ax01[0].set_xlabel('time[sec]')
ax01[0].set_ylabel('reference count')

ax01[1].scatter(np.diff(timercount), np.diff(reference))
ax01[1].set_xlabel('difference time of lines')
ax01[1].set_ylabel('difference counts of lines')

In [ ]:
hoge = np.where((np.diff(timercount) > 101000) | (np.diff(timercount) < 99000))[0]
len(hoge)

In [ ]:
#fig01.savefig(storepath + storename + '_RawAndCorr.png')

In [ ]:
def packet_capture(dataframe, list_ampere, list_interval, UnixTime_Data):
    JST = timezone(timedelta(hours=+9), 'JST')
    initial_UnixTime = []
    switching_UnixTime = []
    start_UnixTime = []

    for i in range(iteration):
        start_UnixTime.append([])

    for i in list_ampere:
        for j in list_interval:
            list_datetime = ((dataframe[dataframe.iloc[:,5] == i])[(dataframe[dataframe.iloc[:,5] == i]).iloc[:,8] == j]).iloc[:,0:2].to_numpy()
            for k in range(iteration):
                if k%pack_size == 0:
                    start_UnixTime[int(k/pack_size)] = (((datetime.strptime(list_datetime[k][0]+' '+list_datetime[k][1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
                    initial_UnixTime = np.append(initial_UnixTime, start_UnixTime[int(k/pack_size)])
                    pass
                pass
            switching_UnixTime.append(min(start_UnixTime))
            pass
        pass

    stop_UnixTime = dataframe[dataframe.iloc[:,2] == 'OFF'].iloc[-1,:]
    end_UnixTime = (((datetime.strptime(stop_UnixTime[0]+' '+stop_UnixTime[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()

    print(((datetime.strptime(stop_UnixTime[0]+' '+stop_UnixTime[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)))
    
    captured_initials = []
    captured_switching = []
    
    for i in range(len(switching_UnixTime)):
        captured_switching.append(max(np.where(time_category[::capture_slice] <= switching_UnixTime[i])[0]))
        pass
    
    for i in range(len(initial_UnixTime)):
        captured_initials.append(max(np.where(time_category[::capture_slice] <= initial_UnixTime[i])[0]))
        pass
    
    print(f'{end_UnixTime + pos_roll}, {time_category[::capture_slice][-1]}')
    captured_end = min(np.where(end_UnixTime + pos_roll <= time_category[::capture_slice])[0])
    
    return captured_switching, captured_initials, captured_end

In [ ]:
time_category[-1]-time_category[0]

In [ ]:
reference1 = reference

hoge = np.where(reference > 52000)[0]

for i in hoge:
    reference1[i] = reference[i]-62000

In [ ]:
reference1

In [ ]:
#block
time_block = []
reference_block = []

for i in range(len(swt)-1):
    time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    reference_block.append(reference1[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    pass
time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])
reference_block.append(reference1[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])

lincount_block = []
block_initials = []
onecycle_time = []
proceeded_degrees = []

for i in range(matrix_size):
    lincount_block.append([])
    lincount_block[i] = np.array(np.zeros(len(reference_block[i])))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(reference_block[i]) <= casp_threshold)[0].tolist()

    for j in range(len(reference_block[i])):
        lincount_block[i][j] = reference_block[i][j]+count_offset
        if casp:
            if j == casp[casp_flag]:
                count_offset += reference_block[i][casp[casp_flag]]
                if len(casp) > casp_flag+1:
                    casp_flag += 1
                    pass
                pass
            pass
        pass
    
    block_initials.append([])
    for j in range(len(inits)):
        block_initials[i].append(int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*iteration:(i+1)*iteration])[j]-pos_roll])[0]))
        pass
    
    onecycle_time.append([])
    proceeded_degrees.append([])
    for j in range(len(inits) - 1):
        if j != len(inits) - 1:
            onecycle_time[i].append(time_block[i][block_initials[i][j+1]]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][block_initials[i][j+1]]-lincount_block[i][block_initials[i][j]])*Deg)
            pass
        else:
            onecycle_time[i].append(time_block[i][-1]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][-1]-lincount_block[i][block_initials[i][j]])*Deg)
        pass
    pass

In [ ]:
error_pack = [[0]]
error_flag = 1
UnixInitials = np.array(time_category[::capture_slice][inits], dtype=int)
for i in range(len(inits)):
    #print(f'{error_flag} {error_UnixTime[error_flag]} {UnixInitials[i]}')
    if (i < len(inits)-1) and (error_flag < len(error_UnixTime)-1):
        if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i+1]):
            error_pack[0].append(i)
            #print(f'{error_flag} {error_UnixTime[error_flag]}')
            error_flag += 1
            if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i+1]):
                error_flag += 1
                pass
            pass
        pass
    else:
        if (UnixInitials[i] <= error_UnixTime[error_flag]) and (error_UnixTime[error_flag] < UnixInitials[i]+min(np.diff(UnixInitials))):
            error_pack[0].append(i)
    pass

In [ ]:
len(error_pack[0])

In [ ]:
for i in paku:
    error_pack[0].append(i)

In [ ]:
paku = []

In [ ]:
error_sort = []
error_sort.append([error_pack[0][0]])
for i in sorted(error_pack[0][1:]):
    error_sort[0].append(i)
#error_sort

In [ ]:
len(error_sort[0])

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(len(inits)):
        if i == error_sort[error_flag0][0]:
            if error_flag1 <= len(error_sort[error_flag0])-1:
                if j == error_sort[error_flag0][error_flag1]:
                    #print(f'{error_sort[error_flag0][0]} {error_sort[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_sort)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
#del df

In [ ]:
fig10 = plt.figure(figsize=(10,7))

fuga = []

for i in range(matrix_size):
    fuga.append([])
    for j in range(len(inits)-5):
        fuga[i].append([])
        if error_position[i][j] != False:
            hoge = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]]
            fuga[i][j] = min(np.where(hoge >= 10)[0])
            plt.axvline(hoge[fuga[i][j]])
            plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                        np.diff((reference_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])*Deg,label=f'num_{j}')
            pass
        pass
    pass
plt.ylim([-1,4])

In [ ]:
fig11, ax11 = plt.subplots(1,1,figsize=(300,7))

for i in range(matrix_size):
    for j in range(len(inits)-1):
        #if error_position[i][j] != False:
        ax11.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]),label=f'num_{j}')
            #pass
        pass
    pass

for i in range(len(error_Time)):
    ax11.axvline(error_Time[i], ls=':', lw=3)
    pass

fuga = reference > 60000
ax11.scatter((time[:-1])[~fuga[1:] & fuga[:-1]]-10, np.ones(len((time[:-1])[~fuga[1:] & fuga[:-1]]))*100)

ax11.set_ylim([-1,200])
#ax11.xlim([700,770])

ax11.xaxis.set_major_locator(tik.MultipleLocator(100))

ax11.set_title(f'Jan13_absolute_feedback_5times')
ax11.set_xlabel('time[sec]')
ax11.set_ylabel('difference of counts')

In [ ]:
fig11.savefig(storepath + storename + '_DiffPlots.png')

In [ ]:
cp0 = reference > 60000
cp1 = time[:-1][~cp0[1:] & cp0[:-1]]-10
over_flag = 0

paku = []

for i in range(matrix_size):
    for j in range(len(inits)-1):
        if error_position[i][j] != False:
            cp2 = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]
            #print(f'{over_flag} {round(cp1[over_flag],2)} {round(cp2[0],2)} {round(cp2[-1],2)} {round(cp2[0],2) <=  round(cp1[over_flag],2)} {round(cp1[over_flag],2) < round(cp2[-1],2)}')
            if not (cp2[0] <=  cp1[over_flag]) and (cp1[over_flag] <= cp2[-1]):
                if over_flag < len(cp1)-1:
                    over_flag += 1
                    pass
            if (cp2[0] <=  cp1[over_flag]) and (cp1[over_flag] <= cp2[-1]):
                paku.append(j)
                if over_flag < len(cp1)-1:
                    over_flag += 1
                    pass
                pass
            pass
        pass
    pass

In [ ]:
paku

In [ ]:
for i in range(matrix_size):
    for j in paku:
        if error_position[i][j] != False:
            plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]),label=f'num_{j}')
            pass
        pass
    pass

In [ ]:
fig15 = plt.figure(figsize=(30,7))

for i in range(matrix_size):
    for j in range(450,459):
        if error_position[i][j] != False:
            plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]),label=f'num_{j}')
            pass
        pass
    pass

for i in range(60,70):
    plt.axvline(error_Time[i], ls=':', lw=3)
    pass

fuga = reference > 60000
peke = (time[:-1])[~fuga[1:] & fuga[:-1]]
plt.scatter(peke-10, np.ones(len(peke))*100)

plt.xlim([11700,12300])
plt.ylim([-30,200])
plt.legend()

In [ ]:
moved_angle = []
end_position = []

fig20, ax = plt.subplots(figsize=(13,8))
#ax = axarr.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    end_position.append([])
    for j in range(iteration-1):
        if error_position[i][j] != False:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= 20)[0])
            moved_angle[i].append((reference_block[i][block_initials[i][j] + place]-reference_block[i][block_initials[i][j]])*Deg)
            end_position[i].append(reference_block[i][block_initials[i][j] + place]%52000*Deg)
        pass
        pass
    ax.scatter(end_position[i], moved_angle[i], color='b')
    pass

for i in range(len(np.arange(0,382.5,22.5))):
    ax.axvline(np.arange(0,382.5,22.5)[i], ls=':', color='r')
    pass

#plt.ylim([0,50])

ax.xaxis.set_major_locator(tik.MultipleLocator(22.5))

ax.set_title(f'Jan18_absolute_feedback_8times')
ax.set_xlabel('position[deg]')
ax.set_ylabel('proceeded degree')

In [ ]:
fig20.savefig(storepath + storename + '_POSvsDeg.png', facecolor='white')

In [ ]:
moved_angle = []
end_position = []

fig21, ax = plt.subplots(figsize=(13,8))
#ax = axarr.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    end_position.append([])
    for j in range(iteration-1):
        #if error_position[i][j] != False:
        hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
        place = min(np.where(hoge >= 10)[0])
        moved_angle[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
        end_position[i].append(lincount_block[i][block_initials[i][j] + place]%52000*Deg)
        #pass
        pass
    ax.scatter(end_position[i], moved_angle[i], color='b')
    pass

for i in range(len(np.arange(0,382.5,22.5))):
    ax.axvline(np.arange(0,382.5,22.5)[i], ls=':', color='r')
    pass

#plt.ylim([10,30])

#ax.xaxis.set_major_locator(tik.MultipleLocator(22.5))

ax.set_title(f'Feb16_checkEthe_feedback_5times')
ax.set_xlabel('position[deg]')
ax.set_ylabel('proceeded degree')

In [ ]:
fig21.savefig(storepath + storename + '_POSvsDeg.png', facecolor='white')

In [ ]:
x1 = np.arange(0,360,22.5)
x2 = np.roll(x1, 9)
x3 = np.tile(x2, 14)
x2

In [ ]:
goal = end_position[0]

hoge = np.where(np.array(end_position[0]) > 350)[0]

for i in hoge:
    x3[i] = 360

In [ ]:
len(goal)

In [ ]:
len(x3[:-1])

In [ ]:
plt.scatter(x3[:-1], goal)

In [ ]:
def linear_func(x,a,b):
    return a*x+b

In [ ]:
popt, pcov = curve_fit(linear_func, goal, x3[:-1])
popt

In [ ]:
figX, axarr = plt.subplots(2,1,figsize=(12,16), gridspec_kw={'height_ratios':[3,1]})
plt.rcParams["font.size"] = 25
ax = axarr.ravel()
plt.subplots_adjust(hspace=0.)

piyo = np.linspace(0,360,len(x3[:-1]))

ax[0].scatter(goal, x3[:-1])
ax[0].plot(piyo, linear_func(piyo, popt[0], popt[1]),color='r', lw=1)

hoge = np.arange(0,382.5,22.5)
for i in range(len(hoge)):
    ax[0].axvline(hoge[i], ls=':', color='g')
    ax[0].axhline(hoge[i], ls=':', color='y')

#ax[0].set_xlabel('goal_position[deg]', fontsize=15)
ax[0].set_ylabel('goal position[deg]')
ax[0].yaxis.set_major_locator(tik.MultipleLocator(45))
ax[0].legend(loc='upper left', title=f'gradient a={round(popt[0],5)}\nintercept b={round(popt[1],5)}')

ax[1].hist(goal, bins=360)
ax[1].set_xlabel('stopped position[deg]')
ax[1].set_ylabel('number of events')
ax[1].xaxis.set_major_locator(tik.MultipleLocator(45))

In [ ]:
figX.savefig(storepath + storename + '_PosAccuracy0.png', facecolor='white')

In [ ]:
figX1, axarr = plt.subplots(2,1,figsize=(12,16), gridspec_kw={'height_ratios':[3,1]})
plt.rcParams["font.size"] = 30
ax = axarr.ravel()
plt.subplots_adjust(hspace=0.,left=0.2)

piyo = np.linspace(0,360,len(x3[:-1]))

ax[0].scatter(x3[:-1], goal)
ax[0].plot(piyo, linear_func(piyo, popt[0], popt[1]),color='r', lw=1)

hoge = np.arange(0,382.5,22.5)
for i in range(len(hoge)):
    ax[0].axvline(hoge[i], ls=':', color='g')
    ax[0].axhline(hoge[i], ls=':', color='y')

# ax[0].set_xlabel('goal_position[deg]', fontsize=15)
ax[0].set_ylabel('commanded position[deg]',fontsize=30)
ax[0].yaxis.set_major_locator(tik.MultipleLocator(45))
ax[0].legend(loc='upper left', title=f'gradient a={round(popt[0],5)}\nintercept b={round(popt[1],5)}')

ax[1].scatter(x3[:-1], goal-x3[:-1], s=15)
ax[1].set_xlabel('controlled position[deg]',fontsize=30)
ax[1].set_ylabel('$\Delta x$',fontsize=30)
ax[1].xaxis.set_major_locator(tik.MultipleLocator(45))
ax[1].set_ylim([-4,4])

In [ ]:
figX1.savefig(storepath + storename + '_PosAccuracy1.png', facecolor='white')

In [ ]:
figX3 = plt.figure(figsize=(10,7))
plt.hist(goal-x3[:-1],bins=18,density=True)
plt.xlim([-8,8])

In [ ]:
figX3.savefig(storepath + storename + '_diffhist.png', facecolor='white')

In [ ]:
figX2, axarr = plt.subplots(2,2,figsize=(16,16),gridspec_kw={'height_ratios':[3,1], 'width_ratios':[3,1]})
ax = axarr.ravel()
plt.subplots_adjust(hspace=0.,wspace=0.)
# ax0 = figX2.add_subplot(2,2,1)

ax[0].tick_params(labelbottom=False)
ax[1].tick_params(labelleft=False,labelbottom=False)
ax[3].tick_params(labelleft=False)

piyo = np.linspace(0,360,len(x3[:-1]))

ax[0].scatter(x3[:-1], goal)
ax[0].plot(piyo, linear_func(piyo, popt[0], popt[1]),color='r', lw=1)

hoge = np.arange(0,382.5,22.5)
for i in range(len(hoge)):
    ax[0].axvline(hoge[i], ls=':', color='g')
    ax[0].axhline(hoge[i], ls=':', color='y')

# ax[0].set_xlabel('goal_position[deg]', fontsize=15)
# ax[0].set_ylabel('goal position[deg]')
ax[0].yaxis.set_major_locator(tik.MultipleLocator(45))
ax[0].legend(loc='upper left', title=f'gradient a={round(popt[0],5)}\nintercept b={round(popt[1],5)}')

# ax1 = figX2.add_subplot(2,2,3)

ax[2].scatter(x3[:-1], goal-x3[:-1], s=15)
# ax[1].set_xlabel('stopped position[deg]')
# ax[1].set_ylabel('number of events')
ax[2].xaxis.set_major_locator(tik.MultipleLocator(45))
ax[2].set_ylim([-4,4])

ax[3].hist(goal-x3[:-1],bins=15,orientation="horizontal",density=True)
ax[3].set_ylim([-4,4])
ax[3].xaxis.set_major_locator(tik.FixedLocator([0.1,0.3,0.5,0.7]))
#ax[3].invert_yaxis()

In [ ]:
figX2.savefig(storepath + storename + '_PosAccuracy1_1.png', facecolor='white')

In [ ]:
figXX, axarr = plt.subplots(figsize=(10,10))

piyo = np.linspace(0,360,len(x3[:-1]))

plt.scatter(x3[:-1], goal)
plt.plot(piyo, linear_func(piyo, popt[0], popt[1]),color='r')

plt.xlabel('goal_position[deg]')
plt.ylabel('end_position[deg]')

plt.legend(loc='upper left',fontsize=10,title=f'gradient a={round(popt[0],5)}\nintercept b={round(popt[1],5)}')

In [ ]:
figY, ax = plt.subplots(figsize=(16,8))
plt.rcParams["font.size"] = 18
ax.xaxis.set_major_locator(tik.MultipleLocator(45))

plt.scatter(x3[:-1], goal-x3[:-1], s=15)
plt.ylim([-8,8])
plt.xlabel('goal position[deg]')
plt.ylabel('difference between goal and end position[deg]', fontsize=15)

In [ ]:
figY.savefig(storepath + storename + '_PosAccuracy2.png', facecolor='white')

In [ ]:
figYY, axarr = plt.subplots(2,2,figsize=(20,14))
ax = axarr.ravel()

d0 = 4
piko = int(iteration/d0)

for i in range(d0):
    ax[i].scatter((x3[:-1])[piko*i:piko*(i+1)], (goal-x3[:-1])[piko*i:piko*(i+1)],s=15)
    pass

In [ ]:
figYY.savefig(storepath + storename + '_PosAccuracy3.png', facecolor='white')

In [ ]:
iteration/16

In [ ]:
moved_angle = []
end_position = []
peko = []

fig21, ax = plt.subplots(figsize=(13,8))
#ax = axarr.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    end_position.append([])
    for j in range(210,220):
        if error_position[i][j] != False:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= 18)[0])
            moved_angle[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
            end_position[i].append(lincount_block[i][block_initials[i][j] + place]%52000*Deg)
            peko.append(j)
            pass
        pass
    ax.scatter(end_position[i], moved_angle[i], color='b')
    for l, txt in enumerate(peko):
        ax.annotate(txt, (end_position[i][l], moved_angle[i][l]))
    pass

for i in range(len(np.arange(0,382.5,22.5))):
    ax.axvline(np.arange(0,382.5,22.5)[i], ls=':', color='r')
    pass

#plt.ylim([10,30])

ax.xaxis.set_major_locator(tik.MultipleLocator(30))

ax.set_title(f'Jan19_absolute_feedback_8times')
ax.set_xlabel('position[deg]')
ax.set_ylabel('proceeded degree')

In [ ]:
moved_angle = []
end_position = []
peko = []

p1 = 0
p2 = 518

fig21, ax = plt.subplots(figsize=(13,8))
#ax = axarr.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    end_position.append([])
    for j in range(p1,p2):
        #if error_position[i][j] != False:
        hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
        place = min(np.where(hoge >= 23.5)[0])
        moved_angle[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
        end_position[i].append(lincount_block[i][block_initials[i][j] + place]%52000*Deg)
        peko.append(j)
        #pass
        pass
    ax.scatter(end_position[i], moved_angle[i], color='b')
#     for l, txt in enumerate(peko):
#         ax.annotate(txt, (end_position[i][l], moved_angle[i][l]))
    pass

ax.set_ylim([0,30])

for i in range(len(np.arange(0,382.5,22.5))):
    ax.axvline(np.arange(0,382.5,22.5)[i], ls=':', color='r')
    pass

ax.xaxis.set_major_locator(tik.MultipleLocator(30))

ax.set_title(f'Jan14_absolute_feedback_8times')
ax.set_xlabel('position[deg]')
ax.set_ylabel('proceeded degree')

In [ ]:
fig30 = plt.figure(figsize=(10,7))

for i in range(matrix_size):
    plt.hist(moved_angle[i], bins=15)
    pass
plt.title(f'Jan13_absolute_feedback_5times')
plt.xlabel('proceeded degree')    

In [ ]:
fig30.savefig(storepath + storename + '_histDeg.png', facecolor='white')

In [ ]:
movDeg_mean = []
movDeg_stdv = []

for i in range(matrix_size):
    movDeg_mean.append(np.mean(moved_angle[i]))
    movDeg_stdv.append(np.std(moved_angle[i]))
    pass

In [ ]:
fig40 = plt.figure(figsize=(10,7))
plt.rcParams["font.size"] = 18
mu, std = norm.fit(moved_angle)

# Plot the histogram.
plt.hist(moved_angle, bins=30, density=True, alpha=0.3, color='g', label=f'feedback_8times')
plt.legend(fontsize=15,loc='upper right')

# Plot the PDF.
xmin, xmax = plt.xlim([10,35])
plt.ylim([0.,0.7])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'g', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('ratio')

In [ ]:
fig40.savefig(storepath + storename + '_histDEG_FIT0.png', facecolor='white')

In [ ]:
plt.hist(moved_angle, bins=15, density=True, alpha=0.3, color='g', label=f'feedback_5times')

In [ ]:
figXX.savefig(storepath + storename + '_DEGvsSTD.png', facecolor='white')

In [ ]:
kue = np.arange(0,360,22.5)
kue

In [ ]:
len(kue)

In [ ]:
len(lincount_block[0])

In [ ]:
fig10.savefig(storepath + storename +'_speed1-avoidERROR.png', facecolor='white')

In [ ]:
i = 14
for j in range(32,34):
    #if error_position[i][j] != False:
    plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), label=f'num_{j}')
    plt.legend()

In [ ]:
i = 14
j = 20
plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))

In [ ]:
i = 19
j = 20

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:])[fuga]

figY = plt.figure(figsize=(10,7))

plt.plot(piyo-kue, np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), color='k', label='new black belt')
plt.axvline(0.95)
plt.axvline(1.15)
plt.xlabel('time[sec]')
plt.ylabel('difference of counts (~speed)')
plt.xlim([-0.5,2.])
plt.ylim([-20,330])
plt.legend(loc='upper left', fontsize=15)

In [ ]:
proceeded_degrees[i][j]

In [ ]:
figY.savefig(storepath + storename + '_form_21e-1A-1sec_num20.png', transparent=True)

In [ ]:
i = 19
j = 20

toki = np.diff(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))
basho = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1])[fuga]
saku = basho/toki

figY = plt.figure(figsize=(10,7))

plt.plot((piyo-kue)[1:], np.diff(saku)/np.diff(toki), color='k', label='new black belt')
plt.axvline(0)
plt.xlabel('time[sec]')
plt.ylabel('acceleration')
plt.xlim([0.95,1.15])
plt.ylim([-20,1e9])
plt.legend(loc='upper left', fontsize=15)

* <span style="font-size: 200%; color: red;">Here is a check point of analysis.</span>

In [ ]:
fig11 = plt.figure(figsize=(10,10))

i = 0

plt.scatter(np.array(linCount_woErr[i])%52000*Deg,proDeg_woErr[i])
plt.title('3.0A_0.69sec')

In [ ]:
fig11.savefig(storepath + storename + '_POSvsDEG-avoidERROR.png',facecolor='white')

In [ ]:
fig12 = plt.figure(figsize=(10,10))

i = 0

plt.hist(proDeg_woErr[i])
plt.title('3.0A_0.69sec')

In [ ]:
fig12.savefig(storepath + storename + '_histDEG-avoidERROR.png', facecolor='white')

In [ ]:
proDeg_mean = []
proDeg_stdv = []

for i in range(6):
    proDeg_mean.append([])  
    proDeg_stdv.append([])
for i in range(matrix_size):
    proDeg_mean[int(i/6)].append(np.mean(proDeg_woErr[i]))
    proDeg_stdv[int(i/6)].append(np.std(proDeg_woErr[i]))
    pass

In [ ]:
fig14 = plt.figure(figsize=(10,7))
mu, std = norm.fit(proDeg_woErr)

# Plot the histogram.
plt.hist(proDeg_woErr, bins=15, density=True, alpha=0.3, color='r', label='new black belt:3.0A 0.69sec')
plt.legend(fontsize=15,loc='upper left')

# Plot the PDF.
xmin, xmax = plt.xlim([12,34])
plt.ylim([0.,0.18])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
fig14.savefig(storepath + storename + '_histDEG_FIT1.png', transparent=True)

In [ ]:
i = 0
hoge = 10
fuga = 500

time0 = []
time1 = []
time2 = []
time3 = []

for j in range(hoge,fuga):
    time0.append([])
    time1.append([])
    time2.append([])
    time3.append([])
    time0[j-hoge] = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
    time1[j-hoge] = np.where(~time0[j-hoge][:-1] & time0[j-hoge][1:])
    time2[j-hoge] = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:]
    time3[j-hoge] = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:])[time1[j-hoge]]

figY = plt.figure(figsize=(10,7))

for j in range(hoge,fuga):
    plt.plot(time2[j-hoge]-time3[j-hoge], np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), color='k', label='black belt')
    pass

plt.axvline(0.65, color='y')
plt.axvline(0.78, color='b')
plt.axhline(60)
plt.xlabel('time[sec]')
plt.ylabel('difference of counts (~speed)')
plt.xlim([-0.5,2.])
plt.ylim([-20,140])
#plt.legend(loc='upper right', fontsize=15)

In [ ]:
figY.savefig(storepath + storename + '_form_3A-69e-1sec_num10_slice30.png', transparent=True)

In [ ]:
kue = (lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]

In [ ]:
(kue[-1] - kue[0])*Deg

In [ ]:
figXX = plt.figure(figsize=(10,7))
for i in range(6):
    plt.plot(proDeg_mean[i], proDeg_stdv[i], label=f'{round(1.5+i*0.3,2)}A')
    plt.scatter(proDeg_mean[i], proDeg_stdv[i])
    pass
plt.xlabel('respective mean of proceeded degrees[degree]')
plt.ylabel('standard deviation[degree]')
# plt.xlim([-30,670])
# plt.ylim([0,16])
# plt.axvline(360)
plt.legend(loc='upper left', fontsize=15)

In [ ]:
figXX.savefig(storepath + storename + '_DEGvsSTD0.png', facecolor='white')

In [ ]:
i = 12
plt.plot(time_block[i], lincount_block[i])
plt.axvline(time_block[i][block_initials[i][20]])

In [ ]:
linear_refcount = []

specific_refcount = reference_block[12]

for
linear_refcount = np.array(np.zeros(len(specific_refcount)))
    
count_offset = 0
casp_flag = 0
casp = np.where(np.diff(specific_refcount) <= casp_threshold)[0].tolist()

for i in range(len(specific_refcount)):
    linear_refcount[i] = specific_refcount[i]+count_offset
    if casp:
        if i == casp[casp_flag]:
            count_offset += specific_refcount[casp[casp_flag]]
            if len(casp) > casp_flag+1:
                casp_flag += 1
                pass
            pass
        pass
    pass


In [ ]:
len(inits[0:100])

In [ ]:
plt.plot(time_block[12], linear_refcount)
plt.axvline((time[::capture_slice])[(inits[1200:1300])[90]])

In [ ]:
linear_refcount

In [ ]:
hoge = np.where(time_block[12] == (time[::capture_slice])[(inits[1200:1300])[90]])[0]
hoge

In [ ]:
(time_block[12])[hoge]

In [ ]:
i = 3
hoge = int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*100:(i+1)*100])[j]])[0])

In [ ]:
hoge

In [ ]:
lines = []
for i in range(matrix_size):
    lines.append([])
    for j in range(iteration):
        lines[i].append(int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*100:(i+1)*100])[j]])[0]))

In [ ]:
len(lines)

In [ ]:
lines

In [ ]:
(time_block[0])[lines[0][0]]

In [ ]:
time_block[0][0]

In [ ]:
plt.plot(time_block[0], reference_block[0])
plt.axvline((time_block[0])[lines[0][0]])
plt.axvline(time_block[0][0])
plt.xlim([85,90])

In [ ]:
0.08e-3*2.5e-9*100/10e-9

In [ ]:
for i in range(10):
    print(f'{(time[::capture_slice])[swt[i]-1]} {(time[::capture_slice])[swt[i]]} {(time[::capture_slice])[swt[i]+1]}')

In [ ]:
plt.plot(time[::capture_slice], reference[::capture_slice])
plt.xlim([80,100])
plt.ylim([3000,4000])
for i in range(5):
    plt.axvline((time[::capture_slice])[swt[i]-1])
    plt.axvline((time[::capture_slice])[swt[i]])
    plt.axvline((time[::capture_slice])[swt[i]+1])

In [ ]:
plt.plot(time[::capture_slice], reference[::capture_slice])
plt.xlim([580,600])
plt.ylim([3000,4000])
for i in range(5):
    plt.axvline((time[::capture_slice])[swt[i]-1])
    plt.axvline((time[::capture_slice])[swt[i]])
    plt.axvline((time[::capture_slice])[swt[i]+1])

In [ ]:
for i in range(4):
    print(f'{int(np.where(time == (time[::capture_slice])[swt[i]-1])[0])}')

In [ ]:
i = 0
hoge = time[int(np.where(time == (time[::capture_slice])[swt[i]-1])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-1])[0])]


In [ ]:
hoge

In [ ]:
len(time_block)

In [ ]:
for i in range(10):
    print(f'{len(time_block[i])}')

In [ ]:
linear_refcount = np.array(np.zeros(len(specific_refcount)))
    
count_offset = 0
casp_flag = 0
casp = np.where(np.diff(specific_refcount) <= casp_threshold)[0].tolist()

for i in range(len(specific_refcount)):
    linear_refcount[i] = specific_refcount[i]+count_offset
    if casp:
        if i == casp[casp_flag]:
            count_offset += specific_refcount[casp[casp_flag]]
            if len(casp) > casp_flag+1:
                casp_flag += 1
                pass
            pass
        pass
    pass

modLinear = linear_refcount[::skip]
modTime = specific_time[::skip]

positive = np.diff(modLinear) > 0.
edge_initial = np.array([])
edge_initial = np.where(positive[1:] & ~positive[:-1])[0]
edge_final = np.array([])
rep_edge = np.where(~positive[1:] & positive[:-1])[0]
for i in rep_edge:
    edge_flag = 0
    for j in range(edge_coverage):
        if (np.diff(modLinear))[i+j] == 0:
            edge_flag += 1
            if edge_flag == flag_threshold:
                edge_final = np.append(edge_final, i+j+flag_threshold)
                break
                pass
            pass
        pass

In [ ]:
swt

In [ ]:
inits[-1]

In [ ]:
end

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.plot(time, reference, label='non')
#plt.plot(time[::100], reference[::100]+100, label='skip100')
plt.plot(time[::1000], reference[::1000]+200, label='skip1000')
#plt.plot(time[::10000], reference[::10000]+300, label='skip10000')
plt.xlim([22700,22750])
#plt.ylim([0,52000])
for i in range(30):
    plt.axvline((time[::1000])[inits[-i]])
    pass
plt.axvline((time[::1000])[end])
plt.legend(loc='upper left')

In [ ]:
len(swt)

In [ ]:
swt

In [ ]:
time_block = []
reference_block = []

for i in range(len(swt)-1):
    time_block.append((time[::capture_slice])[swt[i]:swt[i+1]])
    reference_block.append((reference[::capture_slice])[swt[i]:swt[i+1]])
    pass
time_block.append((time[::capture_slice])[swt[-1]:end])
reference_block.append((reference[::capture_slice])[swt[-1]:end])

In [ ]:
len(time_block)

In [ ]:
len(reference_block)

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
#del df, initial_lines, time, rep, hoge

# Plots

In [ ]:
fig10 ,axarr10 = plt.subplots(6,6,figsize=(26,28))
ax10 = axarr10.ravel()

for i in range(matrix_size):
    for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
        #if error_position[i][j] != False:
        ax10[i].plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                    (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
        ax10[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
        #ax10[i].xaxis.set_major_locator(tik.MultipleLocator(0.5))

In [ ]:
#fig10.savefig(storepath + storename +'_speed1.png', facecolor='white')

In [ ]:
i = 0

fig = plt.figure(figsize=(10,7))
#for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
for j in range(0,10):
    #if error_position[i][j] != False:
    plt.plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
    plt.title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')

In [ ]:
fig11 ,axarr11 = plt.subplots(6,6,figsize=(24,22))
ax11 = axarr11.ravel()

for i in range(matrix_size):
    ax11[i].scatter(linInitRef_woErr[i],proDeg_woErr[i], s=20)
    ax11[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
    pass

In [ ]:
#fig11.savefig(storepath + storename + '_TvsDEG.png', facecolor='white')

In [ ]:
fig12 = plt.figure(figsize=(10,7))
for i in range(matrix_size):
    plt.scatter(linInitRef_woErr[i], proDeg_woErr[i])
    pass
plt.title('2.1A 0.55sec position VS degrees')
plt.xlabel('position[deg]')
plt.ylabel('proceeded degrees[deg]')

In [ ]:
int(len(linInitRef_woErr[0])/16)

In [ ]:
hoge = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    hoge.append(linInitRef_woErr[0][16*i:16*(i+1)])
    pass
#hoge

In [ ]:
360/23

In [ ]:
plt.hist(linInitRef_woErr[0],range=[0,360],bins=15)

In [ ]:
hist_color = ['bisque', 'lightgreen', 'skyblue']
fuga = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    fuga.append(hist_color[i%3])

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.hist(hoge,range=[0,360],bins=15,stacked=True,color=fuga)

In [ ]:
#fig12.savefig(storepath + storename + '_POSvsDEG.png', facecolor='white')

In [ ]:
fig13 = plt.figure()
for i in range(matrix_size):
    plt.hist(proDeg_woErr[i], bins=15)
    pass
plt.title('2.1A 0.55sec proceeded degrees hist')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig13.savefig(storepath + storename + '_histDEG.png', facecolor='white')

In [ ]:
#Fitting histogram
fig14 = plt.figure(figsize=(10,7))
mu, std = norm.fit(proDeg_woErr)

# Plot the histogram.
plt.hist(proDeg_woErr, bins=15, density=True, alpha=0.3, color='g', label='2.1A 0.55sec')
plt.legend(fontsize=15,loc='upper right')

# Plot the PDF.
xmin, xmax = plt.xlim([0,40])
plt.ylim([0.,0.18])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig14.savefig(storepath + storename + '_histDEG_FIT0.png', facecolor='white')

# Others